In [2]:
import torch
import torch.functional as F
import torch.nn as nn

In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115393])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: tensor([18, 47, 56, 57, 58,  1, 15, 47])
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [9]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 

In [10]:
test=torch.randn((4,8,65))
test = test.reshape((4*8, 65))
test.shape

torch.Size([32, 65])

In [11]:
xb, yb = get_batch('train')
xb

tensor([[ 1, 43, 55, 59, 39, 50,  1, 53],
        [52, 45,  6,  0, 13, 52, 42,  1],
        [41, 47, 43, 58, 63,  6,  0, 32],
        [43, 43, 51, 47, 52, 45,  0, 61]])

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=1)  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

    def start_train(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        for iter in range(10000):
            xb, yb = get_batch('train')
            logits, loss = self(xb, yb)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            if iter % 1000 == 0:
                print(f"iter {iter}: loss {loss.item()}")

# Instantiate and train
m = BigramLanguageModel(vocab_size)
m.start_train()


iter 0: loss 4.784310817718506
iter 1000: loss 4.296200275421143
iter 2000: loss 3.3838226795196533
iter 3000: loss 3.3929059505462646
iter 4000: loss 3.0854015350341797
iter 5000: loss 2.7213728427886963
iter 6000: loss 2.6748268604278564
iter 7000: loss 2.773493766784668
iter 8000: loss 2.93967604637146
iter 9000: loss 2.6800167560577393


In [13]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


y me th'd AUSTowicay grear cfr my thed:
NETI:
TINaiglurbthergneYM.
INAf; ainde g wals slulobus whe




In [14]:
torch.manual_seed(337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [22]:
# xbow = torch.zeros((B,T,C))
# for b in range(B):
#     for t in range(T):
#         xprev = x[b,:t+1] # (t,C)
#         xbow[b,t] = torch.mean(xprev, 0)

1st way of doing it 

In [23]:
# # Doing above in a vectorized way rather than looping 
# #using matrix multiply for a weighted aggregation
# wei = torch.tril(torch.ones(T, T))
# wei = wei / wei.sum(1, keepdim=True)
# xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
# torch.allclose(xbow, xbow2)

2nd way

In [15]:
tril = torch.tril(torch.ones(T, T))
wei  = torch.zeros((T, T))
wei  = wei.masked_fill(tril == 0, float('-inf')) # fill upper triangle with -inf where tril is 0
wei  = F.softmax(wei, dim=1) 
xbow3 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
#torch.allclose(xbow, xbow3)

In [16]:
# xbow is our "context vector" for each position in the sequence and its updated based on the previous tokens in the sequence.
# just like in attention, we can find attention matrix and then dot product it with the input embedding to update that word embedding.

In [17]:
# Now but we dont want each token prediction have equal weights on all tokens so ,

# Single attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False) # (4,8,32 ) @ (32,16) -> (4,8,16)
query = nn.Linear(C, head_size, bias=False) # (4,8,32 ) @ (32,16) -> (4,8,16)
wei = (key(x) @ query(x).transpose(-2, -1)) / C**-0.5  # (4,8,16) @ (4,16,8) -> (4,8,8)
tril = torch.tril(torch.ones(T, T)) 
wei  = wei.masked_fill(tril == 0, float('-inf')) # fill upper triangle with -inf where tril is 0
wei  = F.softmax(wei, dim=1) 
#out = wei @ x  -> we dont use embedding , each head have its own meaning for the word
value = nn.Linear(C, head_size, bias=False) 
v = value(x) # (4,8,32 ) @ (32,16) -> (4,8,16)

out = wei @ v # (4,8,8) @ (4,8,16) -> (4,8,16)


In [55]:
wei.shape

torch.Size([4, 8, 8])

In [56]:
out.shape

torch.Size([4, 8, 16])

In [ ]:
total = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Params: {total}")
